In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from compositions import alfa
from sklearn.decomposition import PCA

# Hàm biến đổi ilr
def ilr_transform(data):
    ilr_data = np.zeros_like(data)
    for i in range(1, data.shape[1]):
        ilr_data[:, i - 1] = np.sqrt(i / (i + 1)) * np.log(data[:, i] / data[:, i - 1])
    return ilr_data

# Hàm thực hiện hồi quy tuyến tính với ILR
def ilr_linear_regression(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

# Thực hiện nghiên cứu mô phỏng
def simulation_study(data, alpha_values, n_pcs):
    results = []
    for alpha in alpha_values:
        for n_pc in n_pcs:
            # Biến đổi ILR cho dữ liệu
            X = ilr_transform(data)
            # Áp dụng phương pháp Alpha-PCR với ILR
            xa = alfa(X, alpha)["aff"]
            pca = PCA(n_components=n_pc)
            sc = pca.fit_transform(xa)
            y = data[:, 0]  # Giả sử cột đầu tiên là biến phụ thuộc
            X_train, X_test, y_train, y_test = train_test_split(sc, y, test_size=0.2, random_state=42)
            rmse = ilr_linear_regression(X_train, X_test, y_train, y_test)
            results.append((alpha, n_pc, rmse))
    return results

# Đọc dữ liệu từ tệp CSV và tiền xử lý
def preprocess_data(file_path):
    # Đọc dữ liệu từ tệp CSV
    df = pd.read_csv(file_path)

    # Loại bỏ các giá trị thiếu
    df.dropna(inplace=True)

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)

    return scaled_data

# Thực hiện nghiên cứu mô phỏng và in kết quả
file_path = "data.csv"
data = preprocess_data(file_path)
alpha_values = np.linspace(0.1, 1, 10)
n_pcs = [1, 2, 3, 4, 5, 6, 7]
results = simulation_study(data, alpha_values, n_pcs)
for result in results:
    print("Alpha:", result[0], "- Number of PCs:", result[1], "- RMSE:", result[2])


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [15]:
def kl_alfapcr(y, x, covar=None, a=None, k=None, xnew=None, B=1, ncores=1, tol=1e-07, maxiters=50):
    # Tính toán clr của dữ liệu đầu vào x
    z = clr(x)
    n, p = z.shape
    # Kiểm tra nếu số lượng components vượt quá số lượng features
    if k is not None and k > p:
        k = p
    # Sử dụng PCA để tính toán các thành phần chính và giá trị eigen tương ứng
    eig = PCA(n_components=k).fit(z)
    values = eig.explained_variance_
    # Tính tổng tích lũy của các giá trị eigen
    per = np.cumsum(values / np.sum(values))
    # Lấy các thành phần của PCA
    vec = eig.components_.T
    # Biến đổi dữ liệu ban đầu thành không gian của các thành phần chính
    sc = eig.transform(z)
    # Nếu có covariates, nối chúng vào sc
    if covar is not None:
        sc = np.hstack((sc, covar))
    # Nếu có dữ liệu mới đầu vào, tính toán clr cho nó
    if xnew is not None:
        xnew = clr(xnew)
        xnew = np.hstack((np.dot(xnew, vec), covar))
    # Áp dụng hàm kl_compreg từ gói Compositional cho dữ liệu đã được biến đổi
    return kl_compreg(y, sc, xnew=xnew, B=B, ncores=ncores, tol=tol, maxiters=maxiters)


In [16]:
def kl_alfapcr_tune(y, x, covar=None, M=10, maxk=50, a=np.arange(-1, 1.1, 0.1), mat=None, graph=False, tol=1e-07, maxiters=50):
    n, p = x.shape
    # Kiểm tra nếu giá trị nhỏ nhất trong dữ liệu là 0, chỉ sử dụng các giá trị alpha dương
    if np.min(x.values) == 0:
        a = a[a > 0]
    # Đảm bảo không quá số lượng features khi thiết lập maxk
    if maxk > p:
        maxk = p
    if covar is not None:
        covar = np.array(covar)
    # Tạo ma trận mat để chia dữ liệu thành các tập con
    if mat is None:
        nu = np.random.choice(np.arange(n), size=min(n, round(n / M) * M), replace=False)
        mat = nu.reshape((-1, M))
    mspe = []
    # Lặp qua các giá trị alpha
    for alpha in a:
        xa = clr(x.values)
        msp = np.zeros((M, maxk))
        # Lặp qua từng tập con
        for vim in range(M):
            # Chia dữ liệu thành tập train và tập test
            ytest = y.iloc[mat[:, vim], :]
            ytrain = y.drop(index=y.index[mat[:, vim]])
            xtrain = xa[~np.isin(np.arange(n), mat[:, vim]), :]
            xtest = xa[np.isin(np.arange(n), mat[:, vim]), :]
            # Tính tổng entropy của dữ liệu test
            com = np.sum(ytest * np.log(ytest))
            # Áp dụng PCA cho tập train
            mod = PCA(n_components=maxk).fit(xtrain)
            vec = mod.components_.T
            za = mod.transform(xtrain)
            zanew = np.dot(xtest, vec)
            # Lặp qua từng số lượng components
            for j in range(maxk):
                if covar is not None:
                    # Nếu có covariates, nối chúng vào z
                    z = np.hstack((za[:, :j+1], covar[~np.isin(np.arange(n), mat[:, vim])]))
                    znew = np.hstack((zanew[:, :j+1], covar[np.isin(np.arange(n), mat[:, vim])]))
                else:
                    z = za[:, :j+1]
                    znew = zanew[:, :j+1]
                # Áp dụng hàm kl_compreg từ gói Compositional cho dữ liệu đã được biến đổi
                est = kl_compreg(ytrain.values, z, xnew=znew, tol=1e-07, maxiters=50)['est']
                res = np.sum(ytest * np.log(est))
                msp[vim, j] = com - res * np.isfinite(res)
        mspe.append(msp)
# Tính toán hiệu suất trung bình của từng alpha và số lượng PCs
    performance = np.mean(mspe, axis=1)
    performance = np.vstack(performance)
    # Xác định alpha và số lượng PCs tốt nhất dựa trên hiệu suất
    best_alpha_idx, best_k_idx = np.unravel_index(np.argmin(performance, axis=None), performance.shape)
    best_alpha = a[best_alpha_idx]
    best_k = best_k_idx + 1  # Vì chỉ số bắt đầu từ 0
    best_performance = np.min(performance)
    # Trả về kết quả tinh chỉnh và hiệu suất
    result = {'mspe': mspe, 'performance': performance, 'best_alpha': best_alpha, 'best_k': best_k, 'best_performance': best_performance}
    if graph:
        # Vẽ biểu đồ contour cho hiệu suất
        plt.figure(figsize=(10, 6))
        plt.contourf(a, np.arange(1, maxk + 1), performance.T)
        plt.colorbar(label='Average KL Divergence')
        plt.xlabel('Alpha values')
        plt.ylabel('Number of PCs')
        plt.title('Performance of KL-ALFAPCR Tuning')
        plt.show()
    return result

In [17]:
df = pd.DataFrame(pd.read_csv("./acs2015_census.csv"))
df = df.dropna()
df.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2


In [18]:
# Xác định các biến thành phần độc lập và biến giải thích
compositional_cols = ['Professional','Service','Office','Construction','Production']

#data = 
var_target = df['Income']
#family = family[(family != 0).all(axis=1)]

# chuyển dữ liệu thành phần
compositional_data = df[compositional_cols]
#compositional_data = compositional_data.div(compositional_data.sum(axis=1), axis=0)
compositional_data.head()

,Professional,Service,Office,Construction,Production
0,34.7,17.0,21.3,11.9,15.2
1,22.3,24.7,21.5,9.4,22.0
2,31.4,24.9,22.1,9.2,12.4
3,27.0,20.8,27.0,8.7,16.4
4,49.6,14.2,18.2,2.1,15.8


In [20]:
# Thay thế các giá trị bằng 0 bằng một epsilon nhỏ
epsilon = 1e-9
compositional_data = compositional_data.replace(0, epsilon)

# Chuyển đổi dữ liệu thành phần bằng centered log-ratio (CLR)
clr_compositional_data = clr(compositional_data)

# Chuyển dữ liệu CLR về dạng DataFrame để tiện sử dụng
#clr_compositional_data = pd.DataFrame(clr_compositional_data, columns=compositional_cols)
clr_compositional_data = pd.DataFrame(clr_compositional_data)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(compositional_data, var_target, test_size=0.2, random_state=42)

kl_alfapcr_tune(compositional_data, var_target)

ValueError: not enough values to unpack (expected 2, got 1)